In [1]:
import numpy as np
np.random.seed(42)
import os
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import missingno as msno
import keras
import keras.backend as K
from keras.layers import *
from keras.models import *
from sklearn import preprocessing
from sklearn import metrics
from sklearn.metrics import classification_report 
from sklearn.svm import SVC, SVR
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from keras.activations import *
import math
from sklearn.preprocessing import StandardScaler
from itertools import product
from sklearn.model_selection import RandomizedSearchCV
from functools import partial
import pickle
import time
import tensorflow as tf
from IPython.core.magic import (register_line_magic,
                                register_cell_magic)

import gplearn
from gplearn.genetic import SymbolicRegressor
from gplearn.genetic import SymbolicTransformer
from gplearn.functions import make_function
from gplearn.fitness import make_fitness
%matplotlib inline

Using TensorFlow backend.


In [2]:
def get_vars():
    
    global df
    global df1
    global df2
    
    df = pd.read_csv("fscore.csv")
    df = df[['cpk_1', 'ldh_1', 'uricac_1', 'k_1', "crat3f", "newarf"]]
    
    matrix = df.values
    for i, iv in enumerate(matrix):
        for j, jv in enumerate(iv):
            if jv.strip() == "":
                if j == matrix.shape[1] - 1 :
                    matrix[i, j] = 0
                else:
                    matrix[i, j] = np.nan
                    
    matrix = matrix.astype(np.float32)
    df = pd.DataFrame(matrix, columns=df.columns)
    df1 = df[['cpk_1', 'ldh_1', 'uricac_1', 'k_1', "newarf"]]
    df2 = df[["crat3f", "newarf"]]

In [3]:
def get_model_for_pred_cret(shape):
    '''
    this model predicts crat3f
    '''
    in1 = Input(shape)
    X = Dense(1, activation=linear)(in1)
    
    model = Model(in1, X)
    return model

In [4]:
def get_model_for_cret(shape):
    '''
    this model predicts arf from crat3f
    '''
    in1 = Input(shape)
    X = Dense(5, activation=linear)(in1)
    X =  PReLU()(X)
    X = Dense(1, activation=sigmoid)(X)
    
    
    model = Model(in1, X)
    
    
    return model

In [5]:
def get_acc(y_true, y_pred):
    
    '''
    gives rec0 and rec1
    '''
    
    arr = metrics.confusion_matrix(y_true, y_pred, labels=[0, 1])
    rec0 = arr[0][0] / (arr[0][0]+arr[0][1])
    rec1 = arr[1][1] / (arr[1][1]+arr[1][0])
    return rec0, rec1

In [6]:
def _fit_acc(y, y_pred, w):
    
    '''
    fitness function for the genetic programming part of the paper
    '''
    
    y_pred = y_pred.astype(np.float32)
    y_pred = (y_pred >= 0).astype(np.float32)
    
    
    
    acc = np.sum((y_pred == y)) / y_pred.shape[0]
    #     we also could have used the following for fitness
    #     rec0, rec1 = get_acc(y, y_pred)
    #     res = acc + rec1 * 1.1
    res = acc
    
    return res


In [7]:
fit_acc = make_fitness(_fit_acc, greater_is_better=True)

In [8]:
def drop_and_clean(data_frame, k):
    '''
    this function is used for making our k fold simpler 
    it receives the data and drops the NaN part of the data set
    also the shape of the data set and the size that each test set should be is computed 
    ind is later used to shuffle our data
    '''
    
    data_frame = data_frame.dropna()
    shape0 = data_frame.shape[0]
    ind = np.arange(shape0)
    size = math.ceil(shape0 / k)
    
    
    return data_frame, shape0, ind, size

In [9]:
def training_procedure_1(data, train, scaler = None, model = None):
    '''
    training procedure : we use one neural network (one shallow neural network) to predict the state of a patient using 
    the predicted or real value of crat3f
    '''
    data = data.copy()
    X = data[:, :-1]
    if train :
        scaler  = StandardScaler().fit(X)
    X = scaler.transform(X)
    y = data[:, -1]
    
    if train:
        model = get_model_for_cret([1])
        model.compile(keras.optimizers.adam(5e-4), loss=keras.losses.binary_crossentropy, metrics=["accuracy"])
        model.fit(X, y, epochs=700, batch_size=8, verbose = 0)
    y_p = model.predict(X) >= 0.5
    rec0, rec1 = get_acc(y, y_p)
    
    return rec0, rec1, scaler, model

In [10]:
def training_procedure_2(data, train, scaler = None, model = None):
    '''
    training procedure : uses genetic programming and data from the first four columns 
    to find the state of the patient without crat3f
    '''
    data = data.copy()
    X = data[:, :-1]
    if train :
        scaler  = StandardScaler().fit(X)
    X = scaler.transform(X)
    y = data[:, -1]
    
    if train:
        model = SymbolicRegressor(function_set=['add', 'sub', 'mul', 'div', "max", "log", "sqrt", "abs"], 
                               metric = fit_acc, verbose = 0,
                          population_size=1000, generations=20,
                          stopping_criteria = 3, random_state = 42, init_depth = (5,15), 
                               tournament_size = 100, const_range=(-1.5, 1.5),p_crossover = 0.85 ,p_subtree_mutation = 0.01,
                              p_hoist_mutation = 0.04, p_point_mutation=0.1, p_point_replace=0.1)
        
        model.fit(X, y)
    y_p = model.predict(X) >= 0
    rec0, rec1 = get_acc(y, y_p)
    
    return rec0, rec1, scaler, model

In [11]:
def training_procedure_3(data, train, scaler = None, model1 = None, model2 = None):
    '''
    training procedure : uses one neural network with two parts two predict the state of the patient
    this function is only used on one data set
    '''
    data = data.copy()
    if train :
        scaler  = StandardScaler().fit(data[:, :-1])
    data[:, :-1] = scaler.transform(data[:, :-1])
    X = data[:, :-2]
    y = data[:, -2]
    
    if train:
        model1 = get_model_for_pred_cret([data.shape[1] - 2])
        model1.compile(keras.optimizers.adam(5e-4), loss=keras.losses.mean_squared_error)
        model1.fit(X, y, epochs=1000, batch_size=8, verbose = 0)
        
    
    y = data[:, -1]
    X = model1.predict(X)
    
    if train :    
        model2 = get_model_for_cret([1])
        model2.compile(keras.optimizers.adam(1e-3), loss=keras.losses.binary_crossentropy, metrics=["accuracy"])
        model2.fit(X, y, epochs=1000, batch_size=8, verbose = 0)
        
        
    y_p = model2.predict(X) >= 0.5
    rec0, rec1 = get_acc(y, y_p)
    
    return rec0, rec1, scaler, model1, model2

In [12]:
def training_procedure_4(data1, data2, train, scaler1 = None, scaler2 = None, model1 = None, model2 = None):
    '''
    training procedure : uses one neural network with two parts two predict the state of the patient
    this function is first uses the first data set to learn how to regress the value of crat3f 
    and then it uses the second data set inorder to predict the state of the paitient
    '''
    data1 = data1.copy()
    data2 = data2.copy()
    
    if(train):
        scaler1 = StandardScaler().fit(data2[:, :-1])
        scaler2 = StandardScaler().fit(data1[:, -2:-1])
    
    data1[:, :-2] = scaler1.transform(data1[:, :-2])
    data2[:, :-1] = scaler1.transform(data2[:, :-1])
    data1[:, -2:-1] = scaler2.transform(data1[:, -2:-1])
    
    X = data1[:, :-2]
    y = data1[:, -2]
    
    if (train):
        model1 = get_model_for_pred_cret([data1.shape[1] - 2])
        model1.compile(keras.optimizers.adam(5e-4), loss=keras.losses.mean_squared_error)
        model1.fit(X, y, epochs=1000, batch_size=8, verbose = 0)
    
    X = model1.predict(data2[:, :-1])
    y = data2[:, -1]
    
    if (train) :    
        model2 = get_model_for_cret([1])
        model2.compile(keras.optimizers.adam(1e-3), loss=keras.losses.binary_crossentropy, metrics=["accuracy"])
        model2.fit(X, y, epochs=1000, batch_size=8, verbose = 0)
    
    
        
    
    
    y_p = model2.predict(X) >= 0.5
    rec0, rec1 = get_acc(y, y_p)
    
    return rec0, rec1, scaler1, scaler2, model1, model2
    
    
    

In [13]:
def training_procedure_5(data, train, scaler = None, model = None):
    '''
    training procedure : uses a neural network and data from the first four columns 
    to find the state of the patient without crat3f
    '''
    data = data.copy()
    X = data[:, :-1]
    if train :
        scaler  = StandardScaler().fit(X)
    X = scaler.transform(X)
    y = data[:, -1]
    
    if train:
        model = get_model_for_cret([data.shape[1]-1])
        model.compile(keras.optimizers.adam(5e-4), loss=keras.losses.binary_crossentropy, metrics=["accuracy"])
        model.fit(X, y, epochs = 1000, verbose = 0, batch_size = 8)
    y_p = model.predict(X) >= 0.5
    rec0, rec1 = get_acc(y, y_p)
    
    return rec0, rec1, scaler, model

In [14]:
def k_fold(model_to_use, k = 10):
    
    np.random.seed(42)
    get_vars()
    global df
    global df1
    global df2
    
    
    df, shape0df, ind, size = drop_and_clean(df, k)
    df1, shape0df1, ind1, size1 = drop_and_clean(df1, k)
    df2, shape0df2, ind2, size2 = drop_and_clean(df2, k)
    
    
    
    
    if model_to_use == 0 :
        np.random.shuffle(ind2)
        data = df2.values
        data = data[ind2]
        function_to_use = training_procedure_1
        sizef = size2
        max_size = df2.shape[0]
    
    if model_to_use == 1 :
        np.random.shuffle(ind)
        data = df.values
        data = data[ind]
        function_to_use = training_procedure_3
        sizef = size
        max_size = df.shape[0]
        
    if model_to_use == 2 :
        function_to_use = training_procedure_4
        np.random.shuffle(ind)
        data = df.values
        data = data[ind]
        sizef = size
        max_size = df.shape[0]
#         add : additional
        np.random.shuffle(ind1)
        data_add = df1.values 
        data_add = data_add[ind1]
        sizef_add = size1
        max_size_add = df1.shape[0]
    if model_to_use in [3, 4] :
        if model_to_use == 3 :
            function_to_use = training_procedure_2
        else :
            function_to_use = training_procedure_5
        np.random.shuffle(ind1)
        data = df1.values 
        data = data[ind1]
        sizef = size1
        max_size = df1.shape[0]
        
    
    trs = []
    tss = []
    mdls = []
    
    for i in range(k):
        
        print("k : ", i)
        
        start = sizef * i
        end = min(max_size, (i + 1) * sizef)
        
        data_train = np.concatenate([data[:start], data[end:]], axis = 0)
        data_test  = data[start:end]
        
        
        if model_to_use == 0 :
            
        
            rec0, rec1, scaler, model = function_to_use(data_train, True)
            trs.append((rec0, rec1))
            rec0, rec1, scaler, model = function_to_use(data_test, False, scaler, model)
            tss.append((rec0, rec1))
            mdls.append(model)
            print("train", trs[-1])
            print("test", tss[-1])
            
        if model_to_use == 1 :
            
            rec0, rec1, scaler, model1, model2 = function_to_use(data_train, True)
            trs.append((rec0, rec1))
            rec0, rec1, scaler, model1, model2 = function_to_use(data_test, False, scaler, model1, model2)
            tss.append((rec0, rec1))
            mdls.append((model1, model2))
            print("train", trs[-1])
            print("test", tss[-1])
            
            
        if model_to_use == 2 :
            
            start_add = sizef_add * i
            end_add = min(max_size_add, (i + 1) * sizef_add)

            data_train_add = np.concatenate([data_add[:start_add], data_add[end_add:]], axis = 0)
            data_test_add  = data_add[start_add:end_add]
            
            rec0, rec1, scaler1, scaler2, model1, model2 = function_to_use(data_train, data_train_add, True)
            trs.append((rec0, rec1))
            rec0, rec1, scaler1, scaler2, model1, model2 = function_to_use(data_test, data_test_add, False, scaler1, scaler2, model1, model2)
            tss.append((rec0, rec1))
            mdls.append((model1, model2))
            print("train", trs[-1])
            print("test", tss[-1])
            
        if model_to_use in [3, 4] :
            
            rec0, rec1, scaler, model = function_to_use(data_train, True)
            trs.append((rec0, rec1))
            rec0, rec1, scaler, model = function_to_use(data_test, False, scaler, model)
            tss.append((rec0, rec1))
            mdls.append(model)
            print("train", trs[-1])
            print("test", tss[-1])
        
    return trs, tss, mdls
        
        
        
        
    
    
    
    
    
    

In [16]:
trains = []
tests  = []
for i in range(5) :
    print("model", i)
    k = 10
    if i == 2 or i == 3 :
        k = 5
    trs, tss, _ = k_fold(i, k)
    trains.append(trs)
    tests.append(tss)
    

model 0
k :  0
train (0.9931972789115646, 0.9642857142857143)
test (1.0, 1.0)
k :  1
train (0.9953917050691244, 0.9682539682539683)
test (1.0, 1.0)
k :  2
train (0.9930555555555556, 0.9692307692307692)
test (1.0, 1.0)
k :  3
train (0.9977064220183486, 0.9672131147540983)
test (0.9591836734693877, 1.0)
k :  4
train (0.9954128440366973, 0.9836065573770492)
test (1.0, 0.8571428571428571)
k :  5
train (0.9931506849315068, 0.9661016949152542)
test (1.0, 1.0)
k :  6
train (0.9954337899543378, 0.9661016949152542)
test (0.9787234042553191, 1.0)
k :  7
train (0.993103448275862, 0.967741935483871)
test (1.0, 1.0)
k :  8
train (0.9930875576036866, 0.9841269841269841)
test (1.0, 0.8)
k :  9
train (0.9931972789115646, 0.9682539682539683)
test (1.0, 1.0)
model 1
k :  0
train (1.0, 1.0)
test (1.0, 1.0)
k :  1
train (1.0, 1.0)
test (1.0, 1.0)
k :  2
train (1.0, 1.0)
test (1.0, 1.0)
k :  3
train (1.0, 1.0)
test (1.0, 1.0)
k :  4
train (0.9917355371900827, 0.9583333333333334)
test (1.0, 1.0)
k :  5
trai

In [21]:
trains = np.array([np.mean(i, axis = 0) for i in trains])

In [23]:
tests = np.array([np.mean(i, axis = 0) for i in tests])

In [24]:
names = ["predicting from crat3f : ", "predicting crat3f using all the information and then diagnosting (using only all complete data) ",
        "predicting crat3f using all the information and then diagnosting (using all data) : ", "predicting using data besides crat3f and using genetic programming : ",
        "predicting using data besides crat3f and using neural networks : "]

In [26]:
print("train accuracy \n")
for i, iv in enumerate(trains) :
    print(names[i])
    print("specificity and sensitivity")
    print(iv)

train accuracy 

predicting from crat3f : 
specificity and sensitivity
[0.99427366 0.97049164]
predicting crat3f using all the information and then diagnosting (using only all complete data) 
specificity and sensitivity
[0.99917355 0.99583333]
predicting crat3f using all the information and then diagnosting (using all data) : 
specificity and sensitivity
[0.99277003 0.92495089]
predicting using data besides crat3f and using genetic programming : 
specificity and sensitivity
[0.99518742 0.90536554]
predicting using data besides crat3f and using neural networks : 
specificity and sensitivity
[1.         0.93329949]


In [27]:
print("train accuracy \n")
for i, iv in enumerate(tests) :
    print(names[i])
    print("specificity and sensitivity")
    print(iv)

train accuracy 

predicting from crat3f : 
specificity and sensitivity
[0.99379071 0.96571429]
predicting crat3f using all the information and then diagnosting (using only all complete data) 
specificity and sensitivity
[0.99375 1.     ]
predicting crat3f using all the information and then diagnosting (using all data) : 
specificity and sensitivity
[0.99512195 0.89285714]
predicting using data besides crat3f and using genetic programming : 
specificity and sensitivity
[0.98      0.9147619]
predicting using data besides crat3f and using neural networks : 
specificity and sensitivity
[0.98612836 0.91666667]
